<h3>Ingliskeelsete RoBERTa mudelite testimine</h3>

In [12]:
pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from transformers import pipeline

C:\Users\taunotam\Anaconda3\envs\ETP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.loc[df["category"] == 1, "category"] = "POSITIVE"
df.loc[df["category"] == 0, "category"] = "NEUTRAL"
df.loc[df["category"] == -1, "category"] = "NEGATIVE"

C:\Users\taunotam\AppData\Local\Temp\ipykernel_19176\3503521418.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'POSITIVE' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["category"] == 1, "category"] = "POSITIVE"


In [5]:
df = df.dropna(subset=['clean_comment'])
df = df[df['clean_comment'].apply(len) <= 512]

In [6]:
df

,clean_comment,category
0,family mormon have never tried explain them t...,POSITIVE
2,seriously don say thing first all they won get...,NEGATIVE
3,what you have learned yours and only yours wha...,NEUTRAL
5,you should all sit down together and watch the...,NEGATIVE
7,jesus was zen meets jew,NEUTRAL
...,...,...
37244,jesus,NEUTRAL
37245,kya bhai pure saal chutiya banaya modi aur jab...,POSITIVE
37246,downvote karna tha par upvote hogaya,NEUTRAL
37247,haha nice,POSITIVE


In [7]:
text = list(df['clean_comment'])[:1000]
label = list(df['category'])[:1000]

In [12]:
from datasets import load_dataset

In [16]:
dataset = load_dataset('tweet_eval', 'sentiment')

Generating validation split: 100%|██████████████████████████████████████| 2000/2000 [00:00<00:00, 333251.55 examples/s]


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [27]:
dataset["train"]#["text"]

Dataset({
    features: ['text', 'label'],
    num_rows: 45615
})

In [61]:
model_path = "siebert/sentiment-roberta-large-english"
specific_model = pipeline("sentiment-analysis",model=model_path ,tokenizer=model_path)

In [62]:
from tqdm import tqdm
from sklearn.metrics import f1_score

In [63]:
predictions = []
labels = []

for i in tqdm(range(len(dataset["test"]["label"][:2000]))):
    predictions.append(specific_model(dataset["test"]["text"][i])[0]["label"])
    label = dataset["test"]["label"][i]

    if label == 2:
        labels.append("POSITIVE")
    elif label == 1:
        labels.append("NEUTRAL")
    elif label == 0:
        labels.append("NEGATIVE")

print("F1 Score:", f1_score(labels, predictions, average="micro"))

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [15:39<00:00,  2.13it/s]

F1 Score: 0.4565000000000001


In [64]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment"
specific_model = pipeline("sentiment-analysis",model=model_path ,tokenizer=model_path)

In [65]:
predictions = []
labels = []

for i in tqdm(range(len(dataset["test"]["label"][:2000]))):
    predictions.append(specific_model(dataset["test"]["text"][i])[0]["label"])
    label = dataset["test"]["label"][i]
    if label == 2:
        labels.append("LABEL_2")
    elif label == 1:
        labels.append("LABEL_1")
    elif label == 0:
        labels.append("LABEL_0")


print("F1 Score:", f1_score(labels, predictions, average="micro"))

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:33<00:00,  5.99it/s]

F1 Score: 0.7190000000000001


In [66]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
specific_model = pipeline("sentiment-analysis", model=model_path ,tokenizer=model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [67]:
predictions = []
labels = []

for i in tqdm(range(len(dataset["test"]["label"][:2000]))):
    predictions.append(specific_model(dataset["test"]["text"][i])[0]["label"])
    label = dataset["test"]["label"][i]
    if label == 2:
        labels.append("positive")
    elif label == 1:
        labels.append("neutral")
    elif label == 0:
        labels.append("negative")

print("F1 Score:", f1_score(labels, predictions, average="micro"))

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:41<00:00,  5.86it/s]

F1 Score: 0.715


<h3>Proovime teise datasetiga hinnata täpsust</h3>
<h5>Tegmist pole inim-annoteeritud andmestikuga, mistõttu nende tulemuste põhjal mudelit ei vali, kuid huvipärast vaatame tulemusi.</h5>

In [73]:
df = pd.read_csv("reddit_test.csv")

In [74]:
df

,text,label,__index_level_0__
0,once again trump fails see the big picture edu...,1,26561
1,what model did you follow attract youth from r...,1,1801
2,who the this guy the bottom,1,33890
3,still can believe that the hasn addressed the...,2,28759
4,apanasana sort top all course think have past...,1,15710
...,...,...,...
7425,going allahabad day after would anyone like v...,1,24726
7426,results will out today but government formatio...,0,25724
7427,from congress mukt bharat ready reach out the ...,2,25666
7428,ora che penso pisciato mit boston,1,22435


In [75]:
df = df.dropna(subset=['text'])
df = df[df['text'].apply(len) <= 512]

In [76]:
text = list(df['text'])[:1000]
label = list(df['label'])[:1000]

In [77]:
model_path = "siebert/sentiment-roberta-large-english"
specific_model = pipeline("sentiment-analysis",model=model_path ,tokenizer=model_path)

In [78]:
results = []
for i in range(len(text)):
    if specific_model(text[i])[0]["label"] == "POSITIVE" and label[i] == 2:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "NEUTRAL" and label[i] == 1:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "NEGATIVE" and label[i] == 0:
        results.append(1)
    else:
        results.append(0)

print("Täpsus:", results.count(1)/len(results))

Täpsus: 0.374


In [79]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment"
specific_model = pipeline("sentiment-analysis",model=model_path ,tokenizer=model_path)

In [80]:
results = []
for i in range(len(text)):
    if specific_model(text[i])[0]["label"] == "LABEL_2" and label[i] == 2:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "LABEL_1" and label[i] == 1:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "LABEL_0" and label[i] == 0:
        results.append(1)
    else:
        results.append(0)

print("Täpsus:", results.count(1)/len(results))

Täpsus: 0.56


In [81]:
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
specific_model = pipeline("sentiment-analysis", model=model_path ,tokenizer=model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [82]:
results = []
for i in range(len(text)):
    if specific_model(text[i])[0]["label"] == "positive" and label[i] == 2:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "neutral" and label[i] == 1:
        results.append(1)
    elif specific_model(text[i])[0]["label"] == "negative" and label[i] == 0:
        results.append(1)
    else:
        results.append(0)

print("Täpsus:", results.count(1)/len(results))

Täpsus: 0.571
